<a href="https://colab.research.google.com/github/MartinMashalov/ANNSource/blob/master/SARIMA_Binary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yfinance

#NEXT DAY

##SARIMA Next Day

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import accuracy_score
import warnings
from sklearn.metrics import f1_score  # Add this import at the beginning
from statsmodels.tools.sm_exceptions import ValueWarning

warnings.filterwarnings("ignore", category=ValueWarning)

# Download ^GSPC historical data
gspc = yf.Ticker("^GSPC")
df_gspc = gspc.history(period="10y", interval='1d')#[:-1]

df_gspc.interpolate(method='linear', inplace=True)
# Calculate daily returns
df_gspc['returns'] = df_gspc['Close'].pct_change()

# Fit a SARIMA model to the returns
#sarima_model = SARIMAX(df_gspc['returns'].dropna(), order=(1, 0, 1), seasonal_order=(0, 0, 0, 0))
sarima_model = SARIMAX(df_gspc['returns'].dropna(), order=(0, 0, 1), seasonal_order=(1, 0, 1, 12))
sarima_fit = sarima_model.fit(disp=False)

# Make SARIMA predictions
sarima_pred = sarima_fit.predict(start=1)

# Create binary classification: 1 if close is above open, 0 otherwise
actual = (df_gspc['Close'].iloc[1:-1] < df_gspc['Open'].iloc[1:-1]).astype(int).reset_index(drop=True)
predictions = (sarima_pred.values > 0).astype(int)

# Calculate accuracy
accuracy = accuracy_score(actual, predictions)
f1 = f1_score(actual, predictions)
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

# Filter predictions and actual values based on the condition
filtered_indices = np.where((sarima_pred.values[:-2] > 0.002) | (sarima_pred.values[:-2] < -0.002))[0]
filtered_actual = actual.iloc[filtered_indices]
filtered_predictions = predictions[filtered_indices]

# Calculate accuracy for filtered predictions
if len(filtered_indices) > 0:
    filtered_accuracy = accuracy_score(filtered_actual, filtered_predictions)
    print(f"Accuracy (filtered +/- 0.002): {filtered_accuracy}")
else:
    print("No predictions meet the filter condition.")

# Forecast one step ahead
current_prediction = sarima_fit.forecast(steps=1)

# Print the current prediction
#print(f"Price: {df['Open'].iloc[-1]}")
print(f"Last close price: {df_gspc['Close'].iloc[-1]}")
print(f"Current prediction (return): {current_prediction.values[0]}")

# Determine if the predicted return is positive or negative (1 for positive, 0 for negative)
predicted_direction = (current_prediction.values[0] > 0).astype(int)
print(f"Predicted direction: {predicted_direction}")

Accuracy: 0.8633293603496226
F1 Score: 0.8499127399650961
Accuracy (filtered +/- 0.002): 0.9933333333333333
Last close price: 4137.64013671875
Current prediction (return): 0.00021544832810475336
Predicted direction: 1


##BACKTEST of SARIMA Next Day

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score  # Add this import at the beginning

import warnings
from statsmodels.tools.sm_exceptions import ValueWarning

warnings.filterwarnings("ignore", category=ValueWarning)

# Download ^GSPC historical data
gspc = yf.Ticker("^GSPC")
df_gspc = gspc.history(period="10y")#[:-1]
# Calculate daily returns
df_gspc['returns'] = df_gspc['Close'].pct_change()

# Fit a SARIMA model to the returns
#sarima_model = SARIMAX(df_gspc['returns'].dropna(), order=(1, 0, 1), seasonal_order=(0, 0, 0, 0))
sarima_model = SARIMAX(df_gspc['returns'].dropna(), order=(0, 0, 1), seasonal_order=(1, 0, 1, 12))
sarima_fit = sarima_model.fit(disp=False)

# Make SARIMA predictions
sarima_pred = sarima_fit.predict(start=1)

# Create binary classification: 1 if close is above open, 0 otherwise
actual = (df_gspc['Close'].iloc[1:-1] < df_gspc['Open'].iloc[1:-1]).astype(int).reset_index(drop=True)
predictions = (sarima_pred.values > 0).astype(int)

# Calculate accuracy
accuracy = accuracy_score(actual, predictions)
f1 = f1_score(actual, predictions)
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

# Filter predictions and actual values based on the condition
filtered_indices = np.where((sarima_pred.values[:-2] > 0.002) | (sarima_pred.values[:-2] < -0.002))[0]
filtered_actual = actual.iloc[filtered_indices]
filtered_predictions = predictions[filtered_indices]

# Calculate accuracy for filtered predictions
if len(filtered_indices) > 0:
    filtered_accuracy = accuracy_score(filtered_actual, filtered_predictions)
    print(f"Accuracy (filtered +/- 0.002): {filtered_accuracy}")
else:
    print("No predictions meet the filter condition.")

# Forecast one step ahead
current_prediction = sarima_fit.forecast(steps=1)

# Print the current prediction
#print(f"Price: {df['Open'].iloc[-1]}")
print(f"Last close price: {df_gspc['Close'].iloc[-1]}")
print(f"Current prediction (return): {current_prediction.values[0]}")

# Determine if the predicted return is positive or negative (1 for positive, 0 for negative)
predicted_direction = (current_prediction.values[0] > 0).astype(int)
print(f"Predicted direction: {predicted_direction}")

# ... (rest of your code) ...

# Initialize variables for the backtest
backtest_days = 30
correct_predictions = 0

# Loop through the last 10 days
for i in range(1, backtest_days + 1):
    # Fit a SARIMA model using the data up to the current day
    current_sarima_model = SARIMAX(df_gspc['returns'].iloc[:-i].dropna(), order=(1, 0, 1), seasonal_order=(0, 0, 0, 0))
    current_sarima_fit = current_sarima_model.fit(disp=False)

    # Forecast one step ahead
    current_prediction = current_sarima_fit.forecast(steps=1)
    predicted_direction = (current_prediction.values[0] > 0).astype(int)

    # Compare the predicted direction with the actual direction
    actual_direction = (df_gspc['Close'].iloc[-i - 1] < df_gspc['Open'].iloc[-i - 1]).astype(int)
    correct = predicted_direction == actual_direction
    correct_predictions += correct

    # Print the prediction and whether it was correct
    print(f"Date: {df_gspc.index[-i].strftime('%Y-%m-%d')}")
    print(f"Predicted direction: {predicted_direction}")
    print(f"Actual direction: {actual_direction}")
    print(f"Prediction correct: {correct}")
    print()

# Calculate and print the accuracy of the backtest
backtest_accuracy = correct_predictions / backtest_days
print(f"Backtest accuracy: {backtest_accuracy:.4f}")

Accuracy: 0.8633293603496226
F1 Score: 0.8499127399650961
Accuracy (filtered +/- 0.002): 0.9933333333333333
Last close price: 4137.64013671875
Current prediction (return): 0.00021544832810475336
Predicted direction: 1
Date: 2023-04-14
Predicted direction: 0
Actual direction: 0
Prediction correct: True

Date: 2023-04-13
Predicted direction: 1
Actual direction: 1
Prediction correct: True

Date: 2023-04-12
Predicted direction: 0
Actual direction: 1
Prediction correct: False

Date: 2023-04-11
Predicted direction: 1
Actual direction: 0
Prediction correct: False

Date: 2023-04-10
Predicted direction: 0
Actual direction: 0
Prediction correct: True

Date: 2023-04-06
Predicted direction: 1
Actual direction: 1
Prediction correct: True

Date: 2023-04-05
Predicted direction: 1
Actual direction: 1
Prediction correct: True

Date: 2023-04-04
Predicted direction: 1
Actual direction: 0
Prediction correct: False

Date: 2023-04-03
Predicted direction: 0
Actual direction: 0
Prediction correct: True

Date:

##Search for Optimal Seasonality

###random seach for highest f1 score

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import accuracy_score, f1_score
import warnings
import random
import itertools
from statsmodels.tools.sm_exceptions import ValueWarning

warnings.filterwarnings("ignore", category=ValueWarning)

# Download ^GSPC historical data
gspc = yf.Ticker("^GSPC")
df_gspc = gspc.history(period="1mo")
df_gspc['returns'] = df_gspc['Close'].pct_change()

# Define ranges for p, d, q, P, D, Q, and S parameters
p_range = range(0, 3)
d_range = range(0, 2)
q_range = range(0, 3)
P_range = range(0, 2)
D_range = range(0, 2)
Q_range = range(0, 2)
S_range = range(1, 13)  # Seasonality range, assuming monthly seasonality

# Initialize variables for tracking best parameters
best_params = None
best_seasonal_params = None
best_f1 = -np.inf

# Number of random combinations to test
n_combinations = 50

# Random search
for _ in range(n_combinations):
    p, d, q = random.sample(p_range, 1)[0], random.sample(d_range, 1)[0], random.sample(q_range, 1)[0]
    P, D, Q, S = random.sample(P_range, 1)[0], random.sample(D_range, 1)[0], random.sample(Q_range, 1)[0], random.sample(S_range, 1)[0]

    try:
        # Fit the SARIMA model
        sarima_model = SARIMAX(df_gspc['returns'].dropna(), order=(p, d, q), seasonal_order=(P, D, Q, S))
        sarima_fit = sarima_model.fit(disp=False)

        # Make SARIMA predictions
        sarima_pred = sarima_fit.predict(start=1)

        # Create binary classification
        actual = (df_gspc['Close'].iloc[1:-1] < df_gspc['Open'].iloc[1:-1]).astype(int).reset_index(drop=True)
        predictions = (sarima_pred.values > 0).astype(int)

        # Calculate F1 score
        f1 = f1_score(actual, predictions)

        # Check if the current F1 score is better than the best F1 score
        if f1 > best_f1:
            best_f1 = f1
            best_params = (p, d, q)
            best_seasonal_params = (P, D, Q, S)

    except Exception as e:
        pass

# Print the optimal parameters and the corresponding F1 score
print(f"Optimal order: {best_params}")
print(f"Optimal seasonal_order: {best_seasonal_params}")
print(f"Best F1 Score: {best_f1}")

/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/statespace/sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('

Optimal order: (1, 0, 0)
Optimal seasonal_order: (1, 0, 1, 12)
Best F1 Score: 0.823529411764706


###grid search with objective of highest F1 score (work in progress)


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import accuracy_score, f1_score
import warnings
from statsmodels.tools.sm_exceptions import ValueWarning

warnings.filterwarnings("ignore", category=ValueWarning)

# Download ^GSPC historical data
gspc = yf.Ticker("^GSPC")
df_gspc = gspc.history(period="1mo")
df_gspc['returns'] = df_gspc['Close'].pct_change()

# Define ranges for p, d, q, P, D, Q, and S parameters
p_range = range(0, 3)
d_range = range(0, 2)
q_range = range(0, 3)
P_range = range(0, 2)
D_range = range(0, 2)
Q_range = range(0, 2)
S_range = range(1, 13)  # Seasonality range, assuming monthly seasonality

# Initialize variables for tracking best parameters
best_params = None
best_seasonal_params = None
best_f1 = -np.inf

# Grid search
for p, d, q, P, D, Q, S in itertools.product(p_range, d_range, q_range, P_range, D_range, Q_range, S_range):
    try:
        # Fit the SARIMA model
        sarima_model = SARIMAX(df_gspc['returns'].dropna(), order=(p, d, q), seasonal_order=(P, D, Q, S))
        sarima_fit = sarima_model.fit(disp=False)

        # Make SARIMA predictions
        sarima_pred = sarima_fit.predict(start=1)

        # Create binary classification
        actual = (df_gspc['Close'].iloc[1:-1] < df_gspc['Open'].iloc[1:-1]).astype(int).reset_index(drop=True)
        predictions = (sarima_pred.values > 0).astype(int)

        # Calculate F1 score
        f1 = f1_score(actual, predictions)

        # Check if the current F1 score is better than the best F1 score
        if f1 > best_f1:
            best_f1 = f1
            best_params = (p, d, q)
            best_seasonal_params = (P, D, Q, S)

    except Exception as e:
        pass

# Print the optimal parameters and the corresponding F1 score
print(f"Optimal order: {best_params}")
print(f"Optimal seasonal_order: {best_seasonal_params}")
print(f"Best F1 Score: {best_f1}")

###simple grid search (work in progress)

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import itertools
import warnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tools.sm_exceptions import ConvergenceWarning

# Ignore ConvergenceWarning during the grid search
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UserWarning, message="Non-invertible starting seasonal moving average")

# Download ^GSPC historical data
gspc = yf.Ticker("^GSPC")
df_gspc = gspc.history(period="1mo")

# Calculate daily returns
df_gspc['returns'] = df_gspc['Close'].pct_change()

# Define the ranges for p, d, and q
p_range = range(0, 3)
d_range = range(0, 2)
q_range = range(0, 3)

# Define the ranges for seasonal P, D, and Q
seasonal_p_range = range(0, 2)
seasonal_d_range = range(0, 2)
seasonal_q_range = range(0, 2)
seasonal_s = 4

# Create all possible combinations of the parameters
parameters = list(itertools.product(p_range, d_range, q_range))
seasonal_parameters = list(itertools.product(seasonal_p_range, seasonal_d_range, seasonal_q_range, [seasonal_s]))

# Initialize variables to store the best parameters and the lowest AIC
best_parameters = None
best_seasonal_parameters = None
lowest_aic = float("inf")

# Loop through all possible combinations of parameters
for param, seasonal_param in itertools.product(parameters, seasonal_parameters):
    try:
        # Fit the SARIMA model with the current parameters
        model = SARIMAX(df_gspc['returns'].dropna(), order=param, seasonal_order=seasonal_param)
        results = model.fit(disp=False, maxiter=200)  # Increase maxiter to 200

        # Check if the current model has a lower AIC than the previous best model
        if results.aic < lowest_aic:
            best_parameters = param
            best_seasonal_parameters = seasonal_param
            lowest_aic = results.aic
    except Exception as e:
        # If the model fails to fit, skip this combination of parameters
        continue

# Print the best parameters
print(f"Best order: {best_parameters}")
print(f"Best seasonal order: {best_seasonal_parameters}")
print(f"Lowest AIC: {lowest_aic:.4f}")

/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/statespace/sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/statespace/sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/statespace/sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'
/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/statespace/sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'
/usr/local/lib/python3.9/dist-packages/statsmodels/t

Best order: (0, 0, 0)
Best seasonal order: (0, 0, 0, 4)
Lowest AIC: -136.2194


#OTHER MODELS

##XGBoost Next Day

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.model_selection import TimeSeriesSplit, cross_val_score

# Download hourly ^GSPC historical data
gspc = yf.Ticker("^GSPC")
df_gspc = gspc.history(period="10y")#[:-1]
#df_gspc = gspc.history(period="9mo", interval='1h')[:-1]

df_gspc.interpolate(method='linear', inplace=True)

# Calculate hourly returns
df_gspc['returns'] = df_gspc['Close'].pct_change()

# Compute rolling volatility
volatility_window = 20
df_gspc['volatility'] = df_gspc['returns'].rolling(window=volatility_window).std()

# Define a function to map volatility to lookback length
def volatility_to_lookback(volatility):
    if volatility < 0.01:
        return 30
    elif volatility < 0.02:
        return 20
    else:
        return 10

# Prepare the input and output data for the model
max_lookback = 50
X, y = [], []
for i in range(volatility_window, len(df_gspc) - 1):
    lookback = volatility_to_lookback(df_gspc['volatility'].iloc[i])
    if i - lookback < 0:
        continue
    x_i = df_gspc['returns'].iloc[i - lookback:i].values
    x_i = np.pad(x_i, (max_lookback - len(x_i), 0), 'constant', constant_values=(np.nan,))
    X.append(x_i)
    y.append(df_gspc['returns'].iloc[i + 1] > 0)
X, y = np.array(X), np.array(y)

# Train the model using TimeSeriesSplit and cross-validation
tscv = TimeSeriesSplit(n_splits=5)
model = XGBClassifier()
cv_scores = cross_val_score(model, X, y, cv=tscv, scoring='accuracy')

# Display the accuracy
print(f"Cross-validation accuracy: {np.mean(cv_scores):.4f}")

# Fit the model on the entire dataset
model.fit(X, y)

# Make a prediction for the next hour
lookback_for_prediction = volatility_to_lookback(df_gspc['volatility'].iloc[-1])
next_hour_data = df_gspc['returns'].iloc[-lookback_for_prediction:].values
next_hour_data = np.pad(next_hour_data, (max_lookback - len(next_hour_data), 0), 'constant', constant_values=(np.nan,))
next_hour_prediction_proba = model.predict_proba(next_hour_data.reshape(1, -1))
predicted_direction = "ABOVE (1)" if next_hour_prediction_proba[0, 1] > 0.5 else "BELOW (0)"
current_prediction_proba = max(next_hour_prediction_proba[0, 0], next_hour_prediction_proba[0, 1])
print(f"Predicted direction: {predicted_direction}")

# Print the price used to make the prediction
last_price = df_gspc.iloc[-1]['Close']
print(f"Price of ^GSPC used to make the current prediction: {last_price}")
print(f"Probability of the current prediction: {current_prediction_proba:.4f}")

# Calculate accuracy for predictions over 50% probability
threshold = 0.5
y_pred_proba = model.predict_proba(X)
y_pred = (y_pred_proba[:, 1] > 0.5).astype(int)
high_proba_indices = np.where((y_pred_proba[:, 1] > threshold) | (y_pred_proba[:, 0] > threshold))[0]
filtered_y = y[high_proba_indices]
filtered_y_pred = y_pred[high_proba_indices]

if len(high_proba_indices) > 0:
    filtered_accuracy = accuracy_score(filtered_y, filtered_y_pred)
    print(f"Accuracy (filtered > 50% probability): {filtered_accuracy:.4f}")
else:
    print("No predictions meet the filter condition.")

Cross-validation accuracy: 0.4995
Predicted direction: ABOVE (1)
Price of ^GSPC used to make the current prediction: 4146.22021484375
Probability of the current prediction: 0.7147
Accuracy (filtered > 50% probability): 1.0000


##LSTM Same Day

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense

# Download hourly ^GSPC historical data
gspc = yf.Ticker("^GSPC")
df_gspc = gspc.history(period="10y")

# Calculate hourly returns
df_gspc['returns'] = df_gspc['Close'].pct_change()

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_returns = scaler.fit_transform(df_gspc['returns'].dropna().values.reshape(-1, 1))

# Prepare the input and output data for the GRU
lookback = 10
X, y = [], []
for i in range(lookback, len(scaled_returns)):
    X.append(scaled_returns[i - lookback:i, 0])
    y.append(scaled_returns[i, 0] > scaled_returns[i - 1, 0])
X, y = np.array(X), np.array(y)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Split the data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Create and train the GRU model
#70 for 1H, 100 for 1D
model = Sequential()
model.add(GRU(units=100, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(GRU(units=100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Predict on the test set
y_pred_proba = model.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Calculate accuracy for predictions over 60% probability
threshold = 0.6
filtered_indices = np.where((y_pred_proba > threshold) | (y_pred_proba < (1 - threshold)))[0]
filtered_y_test = y_test[filtered_indices]
filtered_y_pred = y_pred[filtered_indices]

if len(filtered_indices) > 0:
    filtered_accuracy = accuracy_score(filtered_y_test, filtered_y_pred)
    print(f"Accuracy (filtered > 60% probability): {filtered_accuracy}")
else:
    print("No predictions meet the filter condition.")

# Forecast the next hour
next_hour_prediction_proba = model.predict(X[-1].reshape(1, -1, 1))
predicted_direction = (next_hour_prediction_proba > 0.5).astype(int)
print(f"Predicted direction: {predicted_direction}")
print(f"Prediction probability: {next_hour_prediction_proba[0, 0]:.2f}")

# Get the index corresponding to the last data point in the scaled_returns array
last_index = len(scaled_returns) - 1

# Get the original price used to make the current prediction
last_price = df_gspc.iloc[last_index]['Close']
print(f"Price of ^GSPC used to make the current prediction: {last_price}")

Epoch 1/10
63/63 [==============================] - 12s 60ms/step - loss: 0.6941 - accuracy: 0.5070
Epoch 2/10
63/63 [==============================] - 3s 49ms/step - loss: 0.6904 - accuracy: 0.5269
Epoch 3/10
63/63 [==============================] - 2s 38ms/step - loss: 0.6854 - accuracy: 0.5354
Epoch 4/10
63/63 [==============================] - 2s 37ms/step - loss: 0.6624 - accuracy: 0.5972
Epoch 5/10
63/63 [==============================] - 2s 38ms/step - loss: 0.6216 - accuracy: 0.6451
Epoch 6/10
63/63 [==============================] - 3s 50ms/step - loss: 0.5770 - accuracy: 0.7049
Epoch 7/10
63/63 [==============================] - 2s 36ms/step - loss: 0.5564 - accuracy: 0.7173
Epoch 8/10
63/63 [==============================] - 3s 42ms/step - loss: 0.5505 - accuracy: 0.7288
Epoch 9/10
63/63 [==============================] - 3s 39ms/step - loss: 0.5546 - accuracy: 0.7228
Epoch 10/10
16/16 [==============================] - 3s 9ms/step
Accuracy: 0.7390438247011952
Accuracy (filt